# AVITO PROJECT

## Team Member

* Sandikha Rahardi - РИМ 130908

## Load Data

### Import Package

In [151]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split

### Set Dataframe

In [152]:
dataset_filename = "assets/avito_dataset.xlsx"
main_df = pd.read_excel(dataset_filename, sheet_name="Sheet1")

## Data Cleaning

### Important Column

In [153]:
# Define important column to predict the price
important_columns = [
    'Цена', 'Метро', 'Метро2', 'Метро. Время', 
    'Район', 'Сегодня просмотров', 'Всего просмотров', 
    'Продавец. Компания', 'Статус', 'Количество комнат',
    'Общая площадь, число', 'Этаж', 'Жилая площадь, число',
    'Санузел', 'Балкон или лоджия', 'Окна',
    'Высота потолков, м', 'Ремонт', 'Способ продажи', 'Мебель',
    'Вид сделки', 'Техника', 'Тип дома', 'Этажей в доме', 'Год постройки',
    'Пассажирский лифт', 'Парковка', 'Двор', 'Грузовой лифт', 'В доме',
    'Тёплый пол', 'Дополнительно', 'Отделка', 'Название новостройки',
    'Корпус, строение', 'Тип участия', 'Площадь кухни, число', 'Тип комнат'
]
print("Total original column : " , main_df.columns.__len__())
print("Total important column : " , important_columns.__len__())
main_df = main_df[important_columns]

Total original column :  71
Total important column :  38


### Clean the dataset

#### Clean about Metro

In [154]:
main_df['Метро'].dropna(inplace=True)

def preprocess_metro_time(row):
    if(type(row) == str):
        row = row.replace("мин.","").strip()
        if(row.__contains__("от")):
            row = float(row.replace("от","").strip())
        elif(row.__contains__("до")):
            row = float(row.replace("до","").strip())
        elif(row.__contains__("–")):
            row = (int(row.split("–")[0]) + int(row.split("–")[1])) / 2
    return row

main_df['Метро. Время'] = main_df['Метро. Время'].apply(preprocess_metro_time)
mean_metro_time = round(pd.to_numeric(main_df['Метро. Время']).mean(), 1)
main_df['Метро. Время'].fillna(mean_metro_time, inplace=True)

main_df['Метро2'] = main_df['Метро2'].apply(lambda x: "Нет" if pd.isna(x) else "Есть")

### Split Dataset

In [87]:
# Split the data into training (80%), testing (15%), and validation (5%)
train_df, temp_df = train_test_split(main_df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=0.25, random_state=42)

In [88]:
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(15842, 38)
(2970, 38)
(991, 38)


### Devided into Categorical and Numerical dataset

In [89]:
numerical_columns = ['Цена', 'Метро. Время', 'Сегодня просмотров', 'Всего просмотров', 'Количество комнат', 'Общая площадь, число', 'Этаж', 'Жилая площадь, число', 'Высота потолков, м', 'Этажей в доме', 'Год постройки', 'Пассажирский лифт', 'Грузовой лифт', 'Площадь кухни, число']
categorical_columns = ['Метро', 'Метро2', 'Район', 'Продавец. Компания', 'Статус', 'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 'Способ продажи', 'Мебель', 'Вид сделки', 'Техника', 'Тип дома', 'Парковка', 'Двор', 'В доме', 'Тёплый пол', 'Дополнительно', 'Отделка', 'Название новостройки', 'Корпус, строение', 'Тип участия', 'Тип комнат']

train_num_df = train_df[numerical_columns]
train_cat_df = train_df[categorical_columns]
test_num_df = test_df[numerical_columns]
test_cat_df = test_df[categorical_columns]
val_num_df = val_df[numerical_columns]
val_cat_df = val_df[categorical_columns]